In [1]:
import numpy as np
import wget
import sacc
import os

ImportError: No module named wget

# Writing SACC files

This notebook illustrates how to create a SACC file from scratch.
For this example we will download the BICEP-Keck-Planck 2018 analysis (BK15, https://arxiv.org/pdf/1810.05216.pdf) and transform it into SACC format.
Note that reading the BK15 data is slightly convoluted, but this is precisely the type of issue SACC is meant to solve. 

## Reading the BK15 data

We start by downloading the data if we don't have it

In [2]:
if not os.path.isfile("BK15_cosmomc/BK15_README.txt"):
    wget.download("http://bicepkeck.org/BK15_datarelease/BK15_cosmomc.tgz")
    os.system("tar -xvf BK15_cosmomc.tgz")
    os.system("rm BK15_cosmomc.tgz")

NameError: name 'os' is not defined

These are the names of the 12 different maps that went into the analysis

In [3]:
# Names for the different frequency maps
map_names = np.array(["BK15_95","BK15_150","BK15_220",
                      "W023","P030","W033","P044",
                      "P070","P100","P143","P217","P353"])

We start by reading all the **bandpasses**

In [5]:
bandpasses = {}
for n in map_names:
    fname = "BK15_cosmomc/data/BK15/bandpass_"+n+".txt"
    bandpasses[n] = np.loadtxt(fname, unpack=True)

IOError: BK15_cosmomc/data/BK15/bandpass_BK15_95.txt not found.

Now we read the **data vector** (i.e. the array of D_ell values):

In [7]:
dv = np.loadtxt("BK15_cosmomc/data/BK15/BK15_cl_hat.dat",unpack=True)[1:]
ncls, nells = dv.shape

IOError: BK15_cosmomc/data/BK15/BK15_cl_hat.dat not found.

Now we read all the **bandpower windows**. Note that they are all sampled on the same ell values.

In [8]:
ls = np.loadtxt("BK15_cosmomc/data/BK15/windows/BK15_bpwf_bin1.txt",unpack=True)[0]
windows=np.array([np.loadtxt("BK15_cosmomc/data/BK15/windows/BK15_bpwf_bin%d.txt"%(i+1),
                             unpack=True)[1:]
                  for i in range(nells)])

IOError: BK15_cosmomc/data/BK15/windows/BK15_bpwf_bin1.txt not found.

Finally, we read the **covariance matrix** and put it in the right order

In [9]:
cov_raw = np.loadtxt("BK15_cosmomc/data/BK15/BK15_covmat_dust.dat", unpack=True)
cov = np.transpose(cov_raw.reshape([nells,ncls,nells,ncls]),
                   axes=[1,0,3,2]).reshape([nells*ncls,nells*ncls])

IOError: BK15_cosmomc/data/BK15/BK15_covmat_dust.dat not found.

OK, this is the **order** in which all the BK15 data is stored.
Right now we need to read this off a txt file and parse it. Since there are 300 possible cross-correlations, this is a huge kerfuffle, hence SACC.

In [11]:
corr_ordering=np.array([['BK15_95_E','BK15_95_E'],['BK15_95_B','BK15_95_B'],['BK15_150_E','BK15_150_E'],
                        ['BK15_150_B','BK15_150_B'],['BK15_220_E','BK15_220_E'],['BK15_220_B','BK15_220_B'],
                        ['W023_E','W023_E'],['W023_B','W023_B'],['P030_E','P030_E'],['P030_B','P030_B'],
                        ['W033_E','W033_E'],['W033_B','W033_B'],['P044_E','P044_E'],['P044_B','P044_B'],
                        ['P070_E','P070_E'],['P070_B','P070_B'],['P100_E','P100_E'],['P100_B','P100_B'],
                        ['P143_E','P143_E'],['P143_B','P143_B'],['P217_E','P217_E'],['P217_B','P217_B'],
                        ['P353_E','P353_E'],['P353_B','P353_B'],['BK15_95_E','BK15_95_B'],
                        ['BK15_95_B','BK15_150_E'],['BK15_150_E','BK15_150_B'],['BK15_150_B','BK15_220_E'],
                        ['BK15_220_E','BK15_220_B'],['BK15_220_B','W023_E'],['W023_E','W023_B'],
                        ['W023_B','P030_E'],['P030_E','P030_B'],['P030_B','W033_E'],['W033_E','W033_B'],
                        ['W033_B','P044_E'],['P044_E','P044_B'],['P044_B','P070_E'],['P070_E','P070_B'],
                        ['P070_B','P100_E'],['P100_E','P100_B'],['P100_B','P143_E'],['P143_E','P143_B'],
                        ['P143_B','P217_E'],['P217_E','P217_B'],['P217_B','P353_E'],['P353_E','P353_B'],
                        ['BK15_95_E','BK15_150_E'],['BK15_95_B','BK15_150_B'],['BK15_150_E','BK15_220_E'],
                        ['BK15_150_B','BK15_220_B'],['BK15_220_E','W023_E'],['BK15_220_B','W023_B'],
                        ['W023_E','P030_E'],['W023_B','P030_B'],['P030_E','W033_E'],['P030_B','W033_B'],
                        ['W033_E','P044_E'],['W033_B','P044_B'],['P044_E','P070_E'],['P044_B','P070_B'],
                        ['P070_E','P100_E'],['P070_B','P100_B'],['P100_E','P143_E'],['P100_B','P143_B'],
                        ['P143_E','P217_E'],['P143_B','P217_B'],['P217_E','P353_E'],['P217_B','P353_B'],
                        ['BK15_95_E','BK15_150_B'],['BK15_95_B','BK15_220_E'],['BK15_150_E','BK15_220_B'],
                        ['BK15_150_B','W023_E'],['BK15_220_E','W023_B'],['BK15_220_B','P030_E'],
                        ['W023_E','P030_B'],['W023_B','W033_E'],['P030_E','W033_B'],['P030_B','P044_E'],
                        ['W033_E','P044_B'],['W033_B','P070_E'],['P044_E','P070_B'],['P044_B','P100_E'],
                        ['P070_E','P100_B'],['P070_B','P143_E'],['P100_E','P143_B'],['P100_B','P217_E'],
                        ['P143_E','P217_B'],['P143_B','P353_E'],['P217_E','P353_B'],['BK15_95_E','BK15_220_E'],
                        ['BK15_95_B','BK15_220_B'],['BK15_150_E','W023_E'],['BK15_150_B','W023_B'],
                        ['BK15_220_E','P030_E'],['BK15_220_B','P030_B'],['W023_E','W033_E'],['W023_B','W033_B'],
                        ['P030_E','P044_E'],['P030_B','P044_B'],['W033_E','P070_E'],['W033_B','P070_B'],
                        ['P044_E','P100_E'],['P044_B','P100_B'],['P070_E','P143_E'],['P070_B','P143_B'],
                        ['P100_E','P217_E'],['P100_B','P217_B'],['P143_E','P353_E'],['P143_B','P353_B'],
                        ['BK15_95_E','BK15_220_B'],['BK15_95_B','W023_E'],['BK15_150_E','W023_B'],
                        ['BK15_150_B','P030_E'],['BK15_220_E','P030_B'],['BK15_220_B','W033_E'],
                        ['W023_E','W033_B'],['W023_B','P044_E'],['P030_E','P044_B'],['P030_B','P070_E'],
                        ['W033_E','P070_B'],['W033_B','P100_E'],['P044_E','P100_B'],['P044_B','P143_E'],
                        ['P070_E','P143_B'],['P070_B','P217_E'],['P100_E','P217_B'],['P100_B','P353_E'],
                        ['P143_E','P353_B'],['BK15_95_E','W023_E'],['BK15_95_B','W023_B'],
                        ['BK15_150_E','P030_E'],['BK15_150_B','P030_B'],['BK15_220_E','W033_E'],
                        ['BK15_220_B','W033_B'],['W023_E','P044_E'],['W023_B','P044_B'],['P030_E','P070_E'],
                        ['P030_B','P070_B'],['W033_E','P100_E'],['W033_B','P100_B'],['P044_E','P143_E'],
                        ['P044_B','P143_B'],['P070_E','P217_E'],['P070_B','P217_B'],['P100_E','P353_E'],
                        ['P100_B','P353_B'],['BK15_95_E','W023_B'],['BK15_95_B','P030_E'],
                        ['BK15_150_E','P030_B'],['BK15_150_B','W033_E'],['BK15_220_E','W033_B'],
                        ['BK15_220_B','P044_E'],['W023_E','P044_B'],['W023_B','P070_E'],['P030_E','P070_B'],
                        ['P030_B','P100_E'],['W033_E','P100_B'],['W033_B','P143_E'],['P044_E','P143_B'],
                        ['P044_B','P217_E'],['P070_E','P217_B'],['P070_B','P353_E'],['P100_E','P353_B'],
                        ['BK15_95_E','P030_E'],['BK15_95_B','P030_B'],['BK15_150_E','W033_E'],
                        ['BK15_150_B','W033_B'],['BK15_220_E','P044_E'],['BK15_220_B','P044_B'],
                        ['W023_E','P070_E'],['W023_B','P070_B'],['P030_E','P100_E'],['P030_B','P100_B'],
                        ['W033_E','P143_E'],['W033_B','P143_B'],['P044_E','P217_E'],['P044_B','P217_B'],
                        ['P070_E','P353_E'],['P070_B','P353_B'],['BK15_95_E','P030_B'],['BK15_95_B','W033_E'],
                        ['BK15_150_E','W033_B'],['BK15_150_B','P044_E'],['BK15_220_E','P044_B'],
                        ['BK15_220_B','P070_E'],['W023_E','P070_B'],['W023_B','P100_E'],['P030_E','P100_B'],
                        ['P030_B','P143_E'],['W033_E','P143_B'],['W033_B','P217_E'],['P044_E','P217_B'],
                        ['P044_B','P353_E'],['P070_E','P353_B'],['BK15_95_E','W033_E'],['BK15_95_B','W033_B'],
                        ['BK15_150_E','P044_E'],['BK15_150_B','P044_B'],['BK15_220_E','P070_E'],
                        ['BK15_220_B','P070_B'],['W023_E','P100_E'],['W023_B','P100_B'],['P030_E','P143_E'],
                        ['P030_B','P143_B'],['W033_E','P217_E'],['W033_B','P217_B'],['P044_E','P353_E'],
                        ['P044_B','P353_B'],['BK15_95_E','W033_B'],['BK15_95_B','P044_E'],['BK15_150_E','P044_B'],
                        ['BK15_150_B','P070_E'],['BK15_220_E','P070_B'],['BK15_220_B','P100_E'],
                        ['W023_E','P100_B'],['W023_B','P143_E'],['P030_E','P143_B'],['P030_B','P217_E'],
                        ['W033_E','P217_B'],['W033_B','P353_E'],['P044_E','P353_B'],['BK15_95_E','P044_E'],
                        ['BK15_95_B','P044_B'],['BK15_150_E','P070_E'],['BK15_150_B','P070_B'],
                        ['BK15_220_E','P100_E'],['BK15_220_B','P100_B'],['W023_E','P143_E'],['W023_B','P143_B'],
                        ['P030_E','P217_E'],['P030_B','P217_B'],['W033_E','P353_E'],['W033_B','P353_B'],
                        ['BK15_95_E','P044_B'],['BK15_95_B','P070_E'],['BK15_150_E','P070_B'],
                        ['BK15_150_B','P100_E'],['BK15_220_E','P100_B'],['BK15_220_B','P143_E'],
                        ['W023_E','P143_B'],['W023_B','P217_E'],['P030_E','P217_B'],['P030_B','P353_E'],
                        ['W033_E','P353_B'],['BK15_95_E','P070_E'],['BK15_95_B','P070_B'],['BK15_150_E','P100_E'],
                        ['BK15_150_B','P100_B'],['BK15_220_E','P143_E'],['BK15_220_B','P143_B'],
                        ['W023_E','P217_E'],['W023_B','P217_B'],['P030_E','P353_E'],['P030_B','P353_B'],
                        ['BK15_95_E','P070_B'],['BK15_95_B','P100_E'],['BK15_150_E','P100_B'],
                        ['BK15_150_B','P143_E'],['BK15_220_E','P143_B'],['BK15_220_B','P217_E'],
                        ['W023_E','P217_B'],['W023_B','P353_E'],['P030_E','P353_B'],['BK15_95_E','P100_E'],
                        ['BK15_95_B','P100_B'],['BK15_150_E','P143_E'],['BK15_150_B','P143_B'],
                        ['BK15_220_E','P217_E'],['BK15_220_B','P217_B'],['W023_E','P353_E'],
                        ['W023_B','P353_B'],['BK15_95_E','P100_B'],['BK15_95_B','P143_E'],['BK15_150_E','P143_B'],
                        ['BK15_150_B','P217_E'],['BK15_220_E','P217_B'],['BK15_220_B','P353_E'],
                        ['W023_E','P353_B'],['BK15_95_E','P143_E'],['BK15_95_B','P143_B'],
                        ['BK15_150_E','P217_E'],['BK15_150_B','P217_B'],['BK15_220_E','P353_E'],
                        ['BK15_220_B','P353_B'],['BK15_95_E','P143_B'],['BK15_95_B','P217_E'],
                        ['BK15_150_E','P217_B'],['BK15_150_B','P353_E'],['BK15_220_E','P353_B'],
                        ['BK15_95_E','P217_E'],['BK15_95_B','P217_B'],['BK15_150_E','P353_E'],
                        ['BK15_150_B','P353_B'],['BK15_95_E','P217_B'],['BK15_95_B','P353_E'],
                        ['BK15_150_E','P353_B'],['BK15_95_E','P353_E'],['BK15_95_B','P353_B'],
                        ['BK15_95_E','P353_B']])


## Creating a SACC file

Now the fun part. Let's start by creating an empty SACC object.                                                                                                                                                  

In [12]:
s = sacc.Sacc()

NameError: name 'sacc' is not defined

Now we add all frequency maps as individual `tracers`.

In [13]:
for n in map_names:
    # The information about each frequency map will become a 'NuMap' tracer object.
    # These are defined by a bandpass and a beam.
    # Since we don't have beam information, we'll make something up.
    ell = np.linspace(2,1000,10)
    bell = np.ones_like(ell)
    # Bandpass data:
    nu = bandpasses[n][0]
    bnu = bandpasses[n][1]
    # We can also pass additional information, such as bandpass or beam uncertainties.
    # E.g. let's say we have ~1% uncertainties on them.
    bnu_extra = {'error' : 0.01*bnu}
    bell_extra = {'error' : 0.01*bell}
    
    s.add_tracer('NuMap', n,
                 nu=nu, bpss_nu=bnu,  # Bandpass data
                 ell=ell, beam_ell=bell,  # Beam data
                 bpss_extra=bnu_extra,  # Additional bandpass data
                 bell_extra=bell_extra,  # Additional beam data
                 nu_unit='GHz')  # Frequency units

KeyError: 'BK15_95'

Now we add all power spectra one by one

In [14]:
# Now we add all power spectra
for ix, xc in enumerate(corr_ordering):
    t1, t2 = xc
    # Possible data types for power spectra:
    # cl_ee, cl_eb, cl_be, cl_bb
    # Additional ones:
    #  - cl_00 (TT-like, mathematically equivalent to cl_ee)
    #  - cl_0e (TE-like, mathematically equivalent to cl_ee)
    #  - cl_0b (TE-like, mathematically equivalent to cl_eb)
    # There are similar types for real-space correlations.
    data_type = 'cl_'+t1[-1].lower()+t2[-1].lower()

    # Let's compute the effective multipole by averaging over the bandpower
    wins_arr = windows[:, ix, :]
    l_eff = np.sum(wins_arr * ls[None,:], axis=1)/np.sum(wins_arr,axis=1)

    # Define SACC window functions
    wins = [sacc.Window(ls, w) for w in wins_arr]

    # Add power spectrum
    s.add_ell_cl(data_type,  # Data type
                 t1[:-2],  # 1st tracer's name
                 t2[:-2],  # 2nd tracer's name
                 l_eff,  # Effective multipole
                 dv[ix],  # Power spectrum values
                 window=wins)  # Bandpower windows

NameError: name 'windows' is not defined

Finally, we add the covariance matrix.
The covariance must be passed in exactly the same order in which the data vector was added.

In [16]:
s.add_covariance(cov)

NameError: name 's' is not defined

Alrighty, let's save it!

In [17]:
s.save_fits("BK15.fits", overwrite=True)

NameError: name 's' is not defined